## **Preprocessing Dataset**

**Normalisasi keypoints dan ekstraksi data ke bentuk digestible untuk model Transformer**

### *Import Library*

###### import library yang digunakan sepanjang eksekusi program ######


In [2]:
import cv2
import numpy as np
import pandas as pd
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import sys
import urllib
import glob
import json, csv

### *Inisiasi Mediapipe*

###### definisikan variabel global untuk penggunaan utilitas mediapipe berulang


In [3]:
mp_holistic = mp.solutions.holistic # Holistic Keypoints : Pose Tubuh dan Tangan
mp_drawing_styles = mp.solutions.drawing_styles
mp_drawing = mp.solutions.drawing_utils # Utilitas menggambar

***Fungsi-fungsi Ekstraksi dan Normalisasi Koordinat Keypoints***

In [30]:
glob_bbox = None
glob_dist = None

def find_body_centroid(landmarks, main_body):
    if landmarks: ### jika landmark ditemukan
        x_bodies = []
        y_bodies = []
        z_bodies = []
        for i in main_body:
            x_bodies.append(landmarks.landmark[i].x)
            y_bodies.append(landmarks.landmark[i].y)
            z_bodies.append(landmarks.landmark[i].z)
        glob_bbox = [x_bodies, y_bodies, z_bodies]
        return np.average(x_bodies), np.average(y_bodies), np.average(z_bodies)
    else: ### jika landmark tidak ditemukan
        return 0, 0, 0
    
def euclidean(a, b):
    sum_sq = np.sum(np.square(a - b)) ## sumasi dari kedua titik
    euclidean = np.sqrt(sum_sq) ## akar kuadrat dari sumasi

    return euclidean

def pixel_match(coordinates, w, h): # rescaling keypoints menyesuaikan resolusi frame
    x = int(coordinates[0] * w)
    y = int(coordinates[1] * h)
    coordinates[0] = x
    coordinates[1] = y

    return coordinates

def landmarks_data(landmarks, data, key):
    radius = data["radius"]
    px_radius = data["px_radius"]
    centroid = data["centroid"]
    coordinates = []
    centroid_x = centroid[0]
    centroid_y = centroid[1]
    original_h = data["image"].shape[0]
    original_w = data["image"].shape[1]
    px_centroid_x, px_centroid_y = pixel_match([centroid_x, centroid_y], original_w, original_h)
    x = 0
    if landmarks:
        for i in landmarks.landmark:
            horizontal = px_centroid_x - px_radius # selisih kanan/kiri
            vertical = px_centroid_y - px_radius # selisih atas/bawah

            px_x, px_y = pixel_match([i.x, i.y], original_w, original_h)
            
            if px_x >= 0: 
                normalized_px_x = abs(horizontal - px_x) 
            else:  
                normalized_px_x = -abs(horizontal - px_x) 
            if px_y >= 0: 
                normalized_px_y = abs(vertical - px_y) 
            else: 
                normalized_px_y = -abs(vertical - px_y)

            # data["img_pixel"] = np.zeros([10*px_radius,10*px_radius,3],dtype=np.uint8)
            # data["img_pixel"].fill(155) # or img[:] = 155
            
            data["img_pixel"] = data["image"]
            data["img_pixel"] = cv2.circle(data["img_pixel"], (px_x,px_y), radius=5, color=(66,66,245), thickness=3)
            data["img_pixel"] = cv2.circle(data["img_pixel"], (int(centroid_x), int(centroid_y)), radius=5, color=(255,255,255), thickness=5)
            data["img_pixel"] = cv2.circle(data["img_pixel"], (normalized_px_x,normalized_px_y), radius=5, color=(62,184,64), thickness=3)
            data["img_pixel"] = cv2.circle(data["img_pixel"], (int(px_centroid_x),int(px_centroid_y)), radius=5, color=(255,251,28), thickness=5)
            #cv2.imwrite(os.path.join(data["data_save_path"])+'/'+str(data["frame_index"])+'_real_.jpg', data["img_pixel"])
            # from matplotlib import pyplot as plt
            # plt.imshow(data["img_pixel"], interpolation='nearest')
            # plt.show()


            # agar skala tidak melebihi 1.92
            i.x = normalized_px_x / (2 * px_radius) #px_radius nilainya sekitar +-500pixel
            i.y = normalized_px_y / (2 * px_radius)
            # i.x = px_x / (2 * px_radius) #px_radius nilainya sekitar +-500pixel
            # i.y = px_y / (2 * px_radius)
            # horizontal_unpix = horizontal / (2 * px_radius)
            # vertical_unpix =  vertical / (2 * px_radius)

            # print(horizontal_unpix, vertical_unpix)

            
            ### cek px_radius, px_centroid_x, px_centroid_y, horizontal dan vertical
            # print("================================")
            # print(data["radius"], data["centroid"], data["data_save_path"])
            # print(px_radius, px_centroid_x, px_centroid_y, horizontal, vertical)
            # print(normalized_px_x, normalized_px_y, px_x, px_y, i.x, i.y)
            #print(str(px_radius))
            # i.x = px_x
            # i.y = px_y
            # print("pixel_asli : "+str([px_x, px_y])+"pixel_normalized : "+str([normalized_px_x,normalized_px_y])+"ix_iy : "+str([i.x,i.y]))
            # hasilnya
            # pixel_asli : [723, 927] pixel_normalized : [795, 943] ix_iy : [0.7443820238113403, 0.8829588294029236]
            # pixel_asli : [679, 968] pixel_normalized : [751, 984] ix_iy : [0.7031835317611694, 0.9213483333587646]
            # pixel_asli : [659, 1017] pixel_normalized : [731, 1033] ix_iy : [0.6844569444656372, 0.9672284722328186]
            # pixel_asli : [653, 1054] pixel_normalized : [725, 1070] ix_iy : [0.6788389682769775, 1.0018726587295532]
            # pixel_asli : [648, 1081] pixel_normalized : [720, 1097] ix_iy : [0.6741573214530945, 1.0271536111831665]
            # pixel_asli : [691, 1041] pixel_normalized : [763, 1057] ix_iy : [0.7144194841384888, 0.9897003769874573]
            # pixel_asli : [684, 1088] pixel_normalized : [756, 1104] ix_iy : [0.7078651785850525, 1.033707857131958]
            # pixel_asli : [674, 1117] pixel_normalized : [746, 1133] ix_iy : [0.6985018849372864, 1.0608614683151245]
            # pixel_asli : [665, 1138] pixel_normalized : [737, 1154] ix_iy : [0.6900749206542969, 1.0805243253707886]
            # pixel_asli : [713, 1041] pixel_normalized : [785, 1057] ix_iy : [0.7350187301635742, 0.9897003769874573]
            # pixel_asli : [705, 1089] pixel_normalized : [777, 1105] ix_iy : [0.7275280952453613, 1.0346442461013794]
            coordinates.append(i.x)
            coordinates.append(i.y)
            coordinates.append(i.z)
            x += 1
    else: ### kalo landmarks tidak ditemukan kosongkan saja semua keypoints di tubuh/tangan
        if key == "hand":
            vertex_num = 21
        if key == "pose":
            vertex_num = 33
        for i in range(0, vertex_num):
            for i in range(0, 3):
                coordinates.append(0)

    return coordinates, landmarks


def draw_landmarks(image, params, key):
    mp_drawing = params["mp_drawing"]

    # additional codes
    # data_save_path = "./saved_data"
    # if not os.path.exists(data_save_path):
    #     os.makedirs(data_save_path)
    if key == 'ori':
        mp_holistic = mp.solutions.holistic
        mp_drawing.draw_landmarks(image, params["pose_landmarks"], mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(0,0,128), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(0,191,255), thickness=2, circle_radius=4)
                              )
        # cv2.imwrite(os.path.join(params["data_save_path"])+'/'+str(params["frame_index"])+'_original_.jpg', image)
        
    elif key == 'nor':
        mp_holistic = mp.solutions.holistic
        mp_drawing.draw_landmarks(image, params["normalized"], mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(
                                  color=(250,128,114), thickness=2, circle_radius=4),
                              mp_drawing.DrawingSpec(
                                  color=(139,0,0), thickness=2, circle_radius=2)
                              )
        # cv2.imwrite(os.path.join(params["data_save_path"])+'/'+str(params["frame_index"])+'_normalized_.jpg', image)
       
    return image

def final_extract(params):
    result = params["result"]
    left_hand_landmarks = result.left_hand_landmarks
    right_hand_landmarks = result.right_hand_landmarks
    pose_landmarks = params["pose_landmarks"]
    shoulders_centroid = params["shoulders_centroid"]
    hips_centroid = params["hips_centroid"]
    image = params["image"]
    im_h = image.shape[0]
    im_w = image.shape[1]
    # centroid bahu yang direkalulasi piksel resolusi
    point_a = pixel_match(shoulders_centroid.copy(), im_w, im_h)
    # centroid pinggang yang direkalulasi piksel resolusi
    point_b = pixel_match(hips_centroid.copy(), im_w, im_h)
   
    point_a = np.array(point_a)
    point_b = np.array(point_b)
    # print(point_a[0],point_b[0])
    params["image"] = cv2.line(params["image"], (int(point_a[0]), int(point_a[1])), (int(point_b[0]), int(point_b[1])), color=(255,255,255), thickness=3)
    params["image"] = cv2.line(params["image"], (int(shoulders_centroid[0]), int(shoulders_centroid[1])), (int(hips_centroid[0]), int(hips_centroid[1])), color=(255,255,255), thickness=3)

    # radius/jarak piksel sesuai dengan rekalkulasi piksel centroid
    px_radius = int(euclidean(point_a, point_b))
    params["px_radius"] = px_radius
    # if params["radius"] == 0:
    #     params["px_radius_multiplier"] = 0
    # else:
    #     # scaling radius dengan keypoints sebelum dan sesudah penyesuaian resolusi frame
    #     params["px_radius_multiplier"] = px_radius / params["radius"] 
    #     # radius rekalkulasi dibagi dengan radius asli untuk dikalikan dengan setiap keypoints di pose terdeteksi

    left_hand_coordinates, lh_landmarks = landmarks_data(left_hand_landmarks, params, "hand")
    right_hand_coordinates, rh_landmarks = landmarks_data(
        right_hand_landmarks, params, "hand")
    pose_coordinates, p_landmarks = landmarks_data(pose_landmarks, params, "pose")
    coordinates_collection = left_hand_coordinates + \
        right_hand_coordinates + pose_coordinates

    params["normalized"] = p_landmarks
    draw_landmarks(params["image"],params,"nor")

    return coordinates_collection

def find_centroid(landmarks, data):
    indices_a = [11, 12]
    indices_b = [23, 24]
    centroid_a = np.array(find_body_centroid(landmarks, indices_a))
    centroid_b = np.array(find_body_centroid(landmarks, indices_b))
    return centroid_a, centroid_b

def keypoints_check(data):
    # Data
    centroid_indices = [0, 11, 12, 23, 24]

    ### global bounding box berfungsi untuk menjadi perwakilan titik vektor centroid badan
    if data["pose_landmarks"] is None and glob_bbox is None: 
        ### kalau pose tidak ditemukan dan bounding box belum ada
        data["centroid"] = [0, 0, 0]
    elif data["pose_landmarks"] is None and glob_bbox is not None: 
        ### kalau pose tidak ditemukan dan bonding box sudah dibentuk maka yang jadi centroid adalah bounding box frame terakhir
        data["centroid"] = glob_bbox
    else: #### kalau pose ditemukan
        data["centroid"] = find_body_centroid(data["pose_landmarks"], centroid_indices)

    if data["pose_landmarks"] is None and glob_dist is not None:
        ### kalau pose tidak ditemukan dan jarak kamera sudah terisi
        ### digunakan untuk tetap mendapatkan nilai radius walaupun pose tidak terdeteksi. namun wajah atau tangan masih terdeteksi.
        # data["radius"] = glob_dist ## digunakan jika ada bantuan deteksi pose oleh tangan atau wajah
        data["shoulders_centroid"] = [0, 0, 0]
        data["hips_centroid"] = [0, 0, 0]
    else: ### kalau pose ditemukan
        centroid_a, centroid_b = find_centroid(data["pose_landmarks"], data) ## dapatkan centroid bahu dan centroid pinggang
        data["radius"] = euclidean(centroid_a, centroid_b) # radius jarak objek terhadap bounding box
        data["shoulders_centroid"] = centroid_a ## centroid bahu
        data["hips_centroid"] = centroid_b ## centroid pinggang
    return data    


def normalize(holistic, mp_holistic, image, frame_index, save_path):
    mp_drawing = mp.solutions.drawing_utils
    # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # image.flags.writeable = False   
    result = holistic.process(image)  # original result
    original = []
    
    params = {"pose_landmarks": result.pose_landmarks,
              "image": image, 
              "result": result, 
              "mp_drawing": mp_drawing, 
              "frame_index" : frame_index,
              "data_save_path" : save_path
              }
    
    if result.left_hand_landmarks:
        for res in result.left_hand_landmarks.landmark:
            original.append(res.x)
            original.append(res.y)
            original.append(res.z)
    else:
        for i in range(0, 21):
            for i in range(0, 3):
                original.append(0)
    
    if result.right_hand_landmarks:
        for res in result.right_hand_landmarks.landmark:
            original.append(res.x)
            original.append(res.y)
            original.append(res.z)
    else:
        for i in range(0, 21):
            for i in range(0, 3):
                original.append(0)

    if result.pose_landmarks:
        for res in result.pose_landmarks.landmark:
            original.append(res.x)
            original.append(res.y)
            original.append(res.z)
    else:
        for i in range(0, 33):
            for i in range(0, 3):
                original.append(0)
        
    #print(original)

    o = save_path+"/"+str(frame_index)+"_original"
    np.savetxt(o+".csv",original,delimiter=",")
    params = keypoints_check(params)
    draw_landmarks(image,params,"ori")
    # image.flags.writeable = True   
    # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    coordinates = final_extract(params)
  

    return coordinates



In [13]:
import os
os.getcwd()

'E:\\viskom\\ActionDetectionforSignLanguage-main'

# **Ekstraksi Frame**

In [31]:
def scanning(path,action, sequence):
    cap = cv2.VideoCapture(path)
    frame_datas = []
    frame_index = 0
    real_seq = sequence + 1
    # label1 = video_path.rsplit('\\', 1)[0]
    # label3 = video_path.rsplit('\\', 1)[-1]
    # label4 = label3.rsplit('.', 1)[0]
    # print(label1)
    # print(label4)
    # print(label3)



    with mp_holistic.Holistic() as holistic:
        while cap.isOpened():
            ret, frame = cap.read()
            if ret == True:
               
                try:
                    # data_save_path = os.path.join(os.getcwd(), 'Results', label1, label4, str(frame_index))
                    data_save_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_index))
                    if not os.path.exists(data_save_path):
                        os.makedirs(data_save_path)

                    coordinates = normalize(holistic, mp_holistic, frame, frame_index, data_save_path) 

                    f = data_save_path+"/"+str(frame_index)+"_normalized"

                    np.savetxt(f+".csv",coordinates,delimiter=",")
               
                    frame_index += 1
                
                except cv2.error as e:
                    # Handle OpenCV error
                    print("Skipping frame due to OpenCV error:", e)
                    continue
                
                except Exception as e:
                    # Handle other types of errors
                    print("Skipping frame due to error:", e)
                    continue
                
                if cv2.waitKey(1) == ord("q"):
                    break
            else:
                break
    print(str(action)+" "+ str(real_seq)+" / 20 done!")
    cap.release()
    cv2.destroyAllWindows()

    # return frame_datas, frame_index


In [32]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('sign_Data') 

# Actions that we try to detect
actions = np.array(['Akan', 'Anda', 'Apa', 'Atau', 'Baca', 'Bagaimana', 'Bahwa', 'Beberapa', 'Besar',
    'Bisa', 'Buah', 'Dan', 'Dari', 'Dengan', 'Dia', 'Haus', 'Ingin', 'Ini', 'Itu',
    'Jadi', 'Juga', 'Kami', 'Kata', 'Kecil', 'Kumpul', 'Labuh', 'Lain', 'Laku',
    'Lapar', 'Main', 'Makan', 'Masing', 'Mereka', 'Milik', 'Minum', 'Oleh', 'Pada',
    'Rumah', 'Satu', 'Saya', 'Sebagai', 'Tambah', 'Tangan', 'Tetapi', 'Tidak', 'Tiga',
    'Udara', 'Untuk', 'Waktu', 'Yang'])

actions_checkpoint_0 = np.array(['akan', 'anda', 'apa', 'atau', 'baca', 'bagaimana', 'bahwa', 'beberapa','besar'
                   ,'bisa','buah','dan' ,'dari' ,'dengan','dia', ])
actions_checkpoint = np.array([
    'haus', 'ingin', 'ini' ,'itu',
 'jadi', 'juga' ,'kami' ,'kata' ,'kecil', 'kumpul' ,'labuh', 'lain' ,'laku',
 'lapar', 'main', 'makan', 'masing', 'mereka', 'milik', 'minum', 'oleh', 'pada',
 'rumah', 'satu', 'saya', 'sebagai', 'tambah', 'tangan' ,'tetapi', 'tidak', 'tiga',
 'udara', 'untuk', 'waktu', 'yang'])

# Thirty videos worth of data
no_sequences = 20

# Videos are going to be 30 frames in length
sequence_length = 120

# Folder start
start_folder = 30

In [19]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [33]:
import glob
import os
import json

for folder in actions1:
    video_save_path = "E:/dataset/video/"
    video_save_path = os.path.join(video_save_path,str(folder))
    
    # if not os.path.exists(data_save_path):
    #     os.makedirs(data_save_path)

    label_index = 0
    bad_video = []
    for i in glob.glob(video_save_path):
        vid_dir = os.path.join(i, "*.mp4")
        file_index = 0
        for j in glob.glob(vid_dir):
            class_name = folder
            file_name = os.path.basename(os.path.splitext(j)[0])
            # save_path = os.path.join(data_save_path, class_name+"_"+str(file_index)+".json")
            keypoints_data = scanning(j,class_name,file_index)

            file_index += 1
        label_index += 1
    print(str(folder)+" done!\n")

Akan 1 / 20 done!
Akan 2 / 20 done!
Akan 3 / 20 done!
Akan 4 / 20 done!
Akan 5 / 20 done!
Akan 6 / 20 done!
Akan 7 / 20 done!
Akan 8 / 20 done!
Akan 9 / 20 done!
Akan 10 / 20 done!
Akan 11 / 20 done!
Akan 12 / 20 done!
Akan 13 / 20 done!
Akan 14 / 20 done!
Akan 15 / 20 done!
Akan 16 / 20 done!
Akan 17 / 20 done!
Akan 18 / 20 done!
Akan 19 / 20 done!
Akan 20 / 20 done!
Akan done!

Anda 1 / 20 done!
Anda 2 / 20 done!
Anda 3 / 20 done!
Anda 4 / 20 done!
Anda 5 / 20 done!
Anda 6 / 20 done!
Anda 7 / 20 done!
Anda 8 / 20 done!
Anda 9 / 20 done!
Anda 10 / 20 done!
Anda 11 / 20 done!
Anda 12 / 20 done!
Anda 13 / 20 done!
Anda 14 / 20 done!
Anda 15 / 20 done!
Anda 16 / 20 done!
Anda 17 / 20 done!
Anda 18 / 20 done!
Anda 19 / 20 done!
Anda 20 / 20 done!
Anda done!

Apa 1 / 20 done!
Apa 2 / 20 done!
Apa 3 / 20 done!
Apa 4 / 20 done!
Apa 5 / 20 done!
Apa 6 / 20 done!
Apa 7 / 20 done!
Apa 8 / 20 done!
Apa 9 / 20 done!
Apa 10 / 20 done!
Apa 11 / 20 done!
Apa 12 / 20 done!
Apa 13 / 20 done!
Apa 14 / 

In [15]:

bad_video = []
for video_path in videos:
    
    
    label1 = video_path.rsplit('\\', 1)[0]
    label3 = video_path.rsplit('\\', 1)[-1]
    label4 = label3.rsplit('.', 1)[0]
    print(video_path)
    print(label1)
    print(label4)
    print(label3)

    # data = scanning(video)
    # if len(data) == 0:
    #     bad = {
    #         "label_index": video,
    #         "label": os.path.split(video)[0],
    #     }
    #     bad_video.append(bad)
    #     continue

E:/dataset/video/akan\0.mp4
E:/dataset/video/akan
0
0.mp4
E:/dataset/video/akan\1 (2).mp4
E:/dataset/video/akan
1 (2)
1 (2).mp4
E:/dataset/video/akan\1.mp4
E:/dataset/video/akan
1
1.mp4
E:/dataset/video/akan\19.mp4
E:/dataset/video/akan
19
19.mp4
E:/dataset/video/akan\2 (2).mp4
E:/dataset/video/akan
2 (2)
2 (2).mp4
E:/dataset/video/akan\2.mp4
E:/dataset/video/akan
2
2.mp4
E:/dataset/video/akan\3 (2).mp4
E:/dataset/video/akan
3 (2)
3 (2).mp4
E:/dataset/video/akan\3.mp4
E:/dataset/video/akan
3
3.mp4
E:/dataset/video/akan\4 (2).mp4
E:/dataset/video/akan
4 (2)
4 (2).mp4
E:/dataset/video/akan\4.mp4
E:/dataset/video/akan
4
4.mp4
E:/dataset/video/akan\5 (2).mp4
E:/dataset/video/akan
5 (2)
5 (2).mp4
E:/dataset/video/akan\5.mp4
E:/dataset/video/akan
5
5.mp4
E:/dataset/video/akan\6 (2).mp4
E:/dataset/video/akan
6 (2)
6 (2).mp4
E:/dataset/video/akan\6.mp4
E:/dataset/video/akan
6
6.mp4
E:/dataset/video/akan\7.mp4
E:/dataset/video/akan
7
7.mp4
E:/dataset/video/akan\VID_20221204_140903(0).mp4
E:/dat